<a href="https://colab.research.google.com/github/midhunjmes/midhun-26-02-24/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

hi


In [1]:
pip install presidio-analyzer


In [2]:
pip install presidio_anonymizer

In [3]:
!pip install spacy


In [4]:
pip install flair regex

In [5]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 898.3 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
from google.colab import files

uploaded = files.upload()  # Upload your CSV file


Saving sensitive.csv to sensitive.csv


In [4]:
from presidio_analyzer import AnalyzerEngine,PatternRecognizer,Pattern,RecognizerResult
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import re
from flair.data import Sentence
from flair.nn import Classifier

In [6]:
ner_tagger = Classifier.load("flair/ner-english-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


2025-03-11 11:25:53,417 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [7]:
import pandas as pd
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]
}
street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)



provider = NlpEngineProvider(nlp_configuration=nlp_config)
nlp_engine = provider.create_engine()

analyzer=AnalyzerEngine(nlp_engine=nlp_engine)
anonymizer=AnonymizerEngine()
analyzer.registry.add_recognizer(street_recognizer)

def detect_locations_with_flair(text):
    sentence = Sentence(text)
    ner_tagger.predict(sentence)

    location_entities = []
    for entity in sentence.get_spans("ner"):
        if entity.tag in ["LOC","ORG", 'PER']:
          entity_type = "LOCATION" if entity.tag == "LOC" else "ORGANIZATION" if entity.tag == "ORG" else "PERSON"
          location_entities.append(RecognizerResult(
                entity_type=entity_type,
                start=entity.start_position,
                end=entity.end_position,
                score=entity.score
            ))
    return location_entities

def analyze_and_anonymize(text):
    presidio_results = analyzer.analyze(
        text=text,
        entities=["PERSON","EMAIL_ADDRESS", "PHONE_NUMBER", "STREET_ADDRESS"],
        language="en",
    )

    flair_results = detect_locations_with_flair(text)
    print(flair_results)
    print(presidio_results)
    all_results =  flair_results + presidio_results
    print(all_results)

    #---- if presidio wants preference over flair------
    #-----------------------------------------------------------
    # # Merge results, but prefer Presidio when both exist
    # final_results = []

    # # Track entity positions already detected by Presidio
    # presidio_positions = {(res.start, res.end) for res in presidio_results}

    # for res in presidio_results:
    #     final_results.append(res)

    # for res in flair_results:
    # # Only add if Presidio did not already detect an entity in the same position
    #     if (res.start, res.end) not in presidio_positions:
    #        final_results.append(res)
    #-----------------------------------------------------------------------------
    anonymized_text = anonymizer.anonymize(text, all_results).text
    return anonymized_text


text='''John Smith and his friend Sarah Johnson recently moved to 789 Elm Street, San Francisco, CA.
John’s email is john.smith@example.com, and Sarah’s email is sarah.johnson@email.net.
They both work at TechCorp, located at 456 Silicon Ave, San Jose, CA.

If you need to reach John, you can call him at +1-415-555-1234.
Sarah’s phone number is (408) 555-5678.

Their friend David Brown, who lives in New York, NY, often visits them.
David’s contact email is david.brown@business.org, and his phone number is 917-555-7890.

Last summer, they took a trip to Los Angeles and stayed at the Grand Sunset Hotel on 12th Avenue.
For hotel reservations, they called +1 310-555-4321.

During the trip, they met Emily White, a researcher from Chicago, IL.
Emily's email is emily.white@university.edu, and her office number is (312) 555-2468.

Later, they traveled to Miami, FL, where they stayed at 234 Ocean Drive, and visited their old college professor, Dr. Robert Williams, whose number is 786-555-1357.

At the end of their journey, they returned home and continued their work at TechCorp.
They frequently travel between different cities for business meetings in Austin, TX, and Seattle, WA.
'''
texts="I am Eagle "

# anonymized_text = analyze_and_anonymize(texts)
# print(anonymized_text)

def anonymize_csv(input_file, output_file):
    df = pd.read_csv(input_file)  # Load CSV

    for column in df.columns:  # Iterate over each column
        print(df[column])
        df[column] = df[column].astype(str).apply(analyze_and_anonymize)

    df.to_csv(output_file, index=False)  # Save anonymized data
    # print(f"Anonymized file saved as {output_file}")

# Run Anonymization
anonymize_csv("sensitive.csv", "anonymized_data.csv")



John Doe                         john.doe@gmail.com
Sarah Lee                       sarah.lee@email.com
David Brown                david.brown@business.org
Emily White              emily.white@university.edu
Robert Williams     robert.williams@techstartup.com
Name: Name, dtype: object
[]
[type: EMAIL_ADDRESS, start: 1, end: 19, score: 1.0]
[type: EMAIL_ADDRESS, start: 1, end: 19, score: 1.0]
[type: PERSON, start: 1, end: 10, score: 0.9415715932846069, type: ORGANIZATION, start: 11, end: 20, score: 0.7915199398994446]
[type: EMAIL_ADDRESS, start: 1, end: 20, score: 1.0]
[type: PERSON, start: 1, end: 10, score: 0.9415715932846069, type: ORGANIZATION, start: 11, end: 20, score: 0.7915199398994446, type: EMAIL_ADDRESS, start: 1, end: 20, score: 1.0]
[type: ORGANIZATION, start: 13, end: 25, score: 0.9997511506080627]
[type: EMAIL_ADDRESS, start: 1, end: 25, score: 1.0]
[type: ORGANIZATION, start: 13, end: 25, score: 0.9997511506080627, type: EMAIL_ADDRESS, start: 1, end: 25, score: 1.0]
[ty

[]
[type: PHONE_NUMBER, start: 0, end: 12, score: 0.4]
[type: PHONE_NUMBER, start: 0, end: 12, score: 0.4]
John Doe                 123 Main Street
Sarah Lee                 789 Elm Street
David Brown                 456 Oak Road
Emily White        1010 Lake Shore Drive
Robert Williams      777 Innovation Blvd
Name: Phone, dtype: object


[type: LOCATION, start: 4, end: 15, score: 0.9999204874038696]
[type: STREET_ADDRESS, start: 0, end: 15, score: 0.85]
[type: LOCATION, start: 4, end: 15, score: 0.9999204874038696, type: STREET_ADDRESS, start: 0, end: 15, score: 0.85]
[type: LOCATION, start: 4, end: 14, score: 0.9999685883522034]
[type: STREET_ADDRESS, start: 0, end: 14, score: 0.85]
[type: LOCATION, start: 4, end: 14, score: 0.9999685883522034, type: STREET_ADDRESS, start: 0, end: 14, score: 0.85]
[type: LOCATION, start: 4, end: 12, score: 0.9999518394470215]
[type: STREET_ADDRESS, start: 0, end: 12, score: 0.85]
[type: LOCATION, start: 4, end: 12, score: 0.9999518394470215, type: STREET_ADDRESS, start: 0, end: 12, score: 0.85]
[type: LOCATION, start: 5, end: 21, score: 0.9998975594838461]
[]
[type: LOCATION, start: 5, end: 21, score: 0.9998975594838461]
[type: LOCATION, start: 4, end: 19, score: 0.9999250769615173]
[type: STREET_ADDRESS, start: 0, end: 19, score: 0.85]
[type: LOCATION, start: 4, end: 19, score: 0.999

In [8]:
files.download("anonymized_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import json

In [7]:


street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+?,\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)
analyzer.registry.add_recognizer(street_recognizer)


def find_location(text):
  sentense=Sentence(text)
  ner_tagger.predict(sentense)
  location=[]
  for entity in sentense.get_spans("ner"):
    if entity.tag=="LOC":
      location.append({"start":entity.start_position,"end":entity.end_position,"entity_type":"LOCATION","score":entity.score})


def anonymize_json(data):
    if isinstance(data, dict):
        return {key: anonymize_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [anonymize_json(item) for item in data]
    elif isinstance(data, str):
        presidio_result = analyzer.analyze(text=data, entities=["PERSON", "EMAIL_ADDRESS", "PHONE_NUMBER", "STREET_ADDRESS"], language="en")
        flair_results=detect_locations_with_flair(data)
        results=presidio_result+flair_results
        return anonymizer.anonymize(text=data, analyzer_results=results).text if results else data
    else:
        return data
json_data = {
    "name": "Apple",
    "email": "mid@gmail.com",
    "phone": "+911234567890",
    "address": "123 Church Street, Kulathoor",
    "messages": [
        "Hello, My name is Apple!",
        "My email is mid@gmail.com and I live in Kochi."
    ],
    "reviews" : {
        "positive": 19,
        "negative": 1,
        "neutral": 1,
        "person": "James Bond",
        "email": "jamesbond@real.com",
        "place": "ernakulam"
    }
}
anonymized_data = anonymize_json(json_data)
print(json.dumps(anonymized_data, indent=2))

{
  "name": "<ORGANIZATION>",
  "email": "<EMAIL_ADDRESS>",
  "phone": "<PHONE_NUMBER>",
  "address": "<STREET_ADDRESS>, <LOCATION>",
  "messages": [
    "Hello, My name is <PERSON>!",
    "My email is <EMAIL_ADDRESS> and I live in <LOCATION>."
  ],
  "reviews": {
    "positive": 19,
    "negative": 1,
    "neutral": 1,
    "person": "<PERSON>",
    "email": "<EMAIL_ADDRESS>",
    "place": "<LOCATION>"
  }
}


In [11]:
# import re
# from flair.data import Sentence
# from flair.nn import Classifier

# # Load Flair's NER model
# ner_tagger = Classifier.load("flair/ner-english-large")

# # Function to find email addresses
# def find_emails(text):
#     email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
#     return [(match.start(), match.end(), "EMAIL") for match in re.finditer(email_pattern, text)]

# # Function to find phone numbers
# def find_phone_numbers(text):
#     phone_pattern = r"\+?\d{1,3}[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,9}"
#     return [(match.start(), match.end(), "PHONE") for match in re.finditer(phone_pattern, text)]

# # Function to mask sensitive information
# def anonymize_text(text):
#     sentence = Sentence(text)

#     # Run Flair NER on text
#     ner_tagger.predict(sentence)

#     # Collect detected entities
#     entities = [(e.start_position, e.end_position, e.tag) for e in sentence.get_spans("ner")]

#     # Add email and phone number detections
#     entities += find_emails(text)
#     entities += find_phone_numbers(text)


#     # Sort entities by start position (important for replacement)
#     entities.sort()
#     print(entities)
#     # Replace entities with masked values
#     anonymized_text = list(text)
#     for start, end, entity_type in reversed(entities):  # Reverse to avoid index shift
#         anonymized_text[start:end] = f"[{entity_type}]"

#     return "".join(anonymized_text)

# # Example text containing sensitive information
# text = """John Smith lives in Trivandrum, Kerala and works at Google.
# His email is john.smith@example.com, and his phone number is (408) 555-5678.
# He recently moved to 456 Park Avenue, New York, NY."""

# # Mask sensitive details
# anonymized_text = anonymize_text(text)
# print(anonymized_text)


In [12]:
# text="Einstein won the Nobel Prize for his work in physics. He spoke German fluently and lived in Switzerland."
# sentence=Sentence(text)
# ner_tagger.predict(sentence)
# print(sentence)
# print(sentence.get_spans("ner"))